In [1]:
import pandas as pd
from pprint import pprint

In [2]:
import re
float_pattern = re.compile(r'\d+\.\d+')
int_pattern = re.compile(r'\d+')
resource_pattern = re.compile(r'마나|배터리|버블|충격|기공|기력|내공')
archemy_pattern = re.compile(r'\](.*?)Lv\.(\d+)')

In [3]:
def flatten_dict(d, parent_key='', sep='_'):
    items={}
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        try:
            v = eval(v)
        except:
            pass
        
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep))
        else:
            items[new_key] = v
    return items

# ArmoryProfile

In [44]:
df = pd.read_csv('./data/ArmoryProfile.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [45]:
# profile table
sql = """
INSERT INTO Profile (
expeditionLevel, pvpGradeName, townLevel, title, guildMemberGrade, guildName, 
usingSkillPoint, totalSkillPoint,
지성, 담력, 매력, 친절,
severName, characterName, characterLevel, characterClassName, itemAvgLevel, itemMaxLevel)
VALUES (%s, %s, %s, %s, %s, %s, 
%s, %s, 
%s, %s, %s, %s, 
%s, %s, %s, %s, %s, %s);
"""

In [46]:
# stats table
sql2 = """ INSERT INTO Stats(
치명_값, 치명_내실증가량, 치명_치명타_적중률(%),
특화_값, 특화_내실증가량, 특화_각성스킬_피해량(%), 특화_효과1, 특화_효과2, 특화_효과3,
제압_값, 제압_내실증가량, 제압_피해증가량(%),
신속_값, 신속_내실증가량, 신속_공격속도(%), 신속_이동속도(%), 신속_스킬_재사용대기시간_감소율(%),
인내_값, 인내_내실증가량, 인내_물리방어력(%), 인내_마법방어력(%), 인내_보호막효과(%), 인내_생명력_회복효과(%),
숙련_값, 숙련_내실증가량, 숙련_상태이상_공격_지속시간(%), 숙련_상태이상_피해_지속시간(%), 숙련_무력화_피해량(%),
최대생명력_값, 최대생명력_체력, 최대생명력_생명활성력(%),
공격력_값, 공격력_기본공격력, 공격력_증감량)
VALUES (
%s, %s, %s, 
%s, %s, %s, %s, %s, %s,
%s, %s, %s, 
%s, %s, %s, %s, %s, 
%s, %s, %s, %s, %s, %s, 
%s, %s, %s, %s, %s, 
%s, %s, %s, 
%s, %s, %s)
"""


In [47]:
stats = {'characterCode' : df['characterCode'][i],
            '치명_값':None, '치명_내실증가량':None, '치명_치명타_적중률(%)':None,
            '특화_값':None, '특화_내실증가량':None, '특화_각성스킬_피해량(%)':None, '특화_효과1':None,
            '특화_효과2':None, '특화_효과3':None,
            '제압_값':None, '제압_내실증가량':None, '제압_피해증가량(%)':None,
            '신속_값':None, '신속_내실증가량':None, '신속_공격속도(%)':None, '신속_이동속도(%)':None,
            '신속_스킬_재사용대기시간_감소율(%)':None,
            '인내_값':None, '인내_내실증가량': None, '인내_물리방어력(%)':None, '인내_마법방어력(%)':None,
            '인내_보호막효과(%)':None, '인내_생명력_회복효과(%)':None,
            '숙련_값':None, '숙련_내실증가량':None, '숙련_상태이상_공격_지속시간(%)':None,
            '숙련_상태이상_피해_지속시간(%)':None, '숙련_무력화_피해량(%)':None,
            '최대생명력_값':None, '최대생명력_체력':None, '최대생명력_생명활성력(%)':None,
            '공격력_값':None, '공격력_기본공격력':None, '공격력_증감량':None}

In [85]:
for i in range(10000):
    cdf = eval(df['data'][i])
    del cdf['CharacterImage']
    
    # 1. profile
    profile = cdf.copy()
    profile['characterCode'] = df['characterCode'][i]
    profile['지성'] = profile['Tendencies'][0]['Point']
    profile['담력'] = profile['Tendencies'][1]['Point']
    profile['매력'] = profile['Tendencies'][2]['Point']
    profile['친절'] = profile['Tendencies'][3]['Point']
    
    del profile['Tendencies']
    del profile['Stats']
    
    # 2. stats
    for k, v in cdf.items():
        if (k == 'Stats') & (v != None):
            stats['치명_값'] = int(v[0]['Value'])
            stats['치명_내실증가량'] = int(int_pattern.findall(v[0]['Tooltip'][1])[0])
            stats['치명_치명타_적중률(%)'] = float(float_pattern.search(v[0]['Tooltip'][0]).group())

            stats['특화_값'] = int(v[1]['Value'])
            stats['특화_내실증가량'] = int(int_pattern.findall(v[1]['Tooltip'][-2])[0])
            try: # 전직 했을 경우
                stats['특화_각성스킬_피해량(%)'] = float(float_pattern.search(v[1]['Tooltip'][-3]).group())
                stats['특화_효과1'] = v[1]['Tooltip'][0]
                stats['특화_효과2'] =  v[1]['Tooltip'][1]
                stats['특화_효과3'] = None if len(v[1]['Tooltip']) == 5 else v[1]['Tooltip'][2]
            except: # 전직 안했을 경우
                stats['특화_각성스킬_피해량(%)'] = None
                stats['특화_효과1'] = None
                stats['특화_효과2'] =  None
                stats['특화_효과3'] = None
            

            stats['제압_값'] = int(v[2]['Value'])
            stats['제압_내실증가량'] = int(int_pattern.findall(v[2]['Tooltip'][2])[0])
            stats['제압_피해증가량(%)'] = float(float_pattern.search(v[2]['Tooltip'][0]).group())

            stats['신속_값'] = int(v[3]['Value'])
            stats['신속_내실증가량'] =  int(int_pattern.findall(v[3]['Tooltip'][-2])[0])
            stats['신속_공격속도(%)'] = float(float_pattern.search(v[3]['Tooltip'][0]).group())
            stats['신속_이동속도(%)'] = float(float_pattern.search(v[3]['Tooltip'][1]).group())
            stats['신속_스킬_재사용대기시간_감소율(%)'] = float(float_pattern.search(v[3]['Tooltip'][2]).group())

            stats['인내_값'] = int(v[4]['Value'])
            stats['인내_내실증가량'] = int(int_pattern.findall(v[4]['Tooltip'][-2])[0])
            stats['인내_물리방어력(%)'] = float(float_pattern.search(v[4]['Tooltip'][0]).group())
            stats['인내_마법방어력(%)'] = float(float_pattern.search(v[4]['Tooltip'][1]).group())
            stats['인내_보호막효과(%)'] =  float(float_pattern.search(v[4]['Tooltip'][2]).group())
            stats['인내_생명력_회복효과(%)'] = float(float_pattern.search(v[4]['Tooltip'][3]).group())

            stats['숙련_값'] =  int(v[5]['Value'])
            stats['숙련_내실증가량'] = int(re.findall(r'\d+', v[5]['Tooltip'][-2])[0])
            stats['숙련_상태이상_공격_지속시간(%)'] = float(float_pattern.search(v[5]['Tooltip'][0]).group())
            stats['숙련_상태이상_피해_지속시간(%)'] =  float(float_pattern.search(v[5]['Tooltip'][1]).group())
            stats['숙련_무력화_피해량(%)'] = float(float_pattern.search(v[5]['Tooltip'][2]).group())

            stats['최대생명력_값'] = int(v[6]['Value'])
            stats['최대생명력_체력'] = int(int_pattern.findall(v[6]['Tooltip'][1])[0])
            stats['최대생명력_생명활성력(%)'] = float(float_pattern.search(v[6]['Tooltip'][2]).group())

            stats['공격력_값'] = int(v[7]['Value'])
            stats['공격력_기본공격력'] =  int(int_pattern.findall(v[7]['Tooltip'][1])[0])
            stats['공격력_증감량'] = int(int_pattern.findall(v[7]['Tooltip'][2])[0])
            



# ArmoryAvatars

In [93]:
df = pd.read_csv('./data/ArmoryAvatars.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [94]:
avatar_parts = ['무기1', '무기2', '머리', '상의1', '상의2', '하의1', '하의2', '얼굴1', '얼굴2', '악기', '이동효과'] 
dic_keys = ['grade', 'isInner', 'isSet', 'name', 'avatarType', 'avatarType2', 'availableClass', 
            'availableTrade', 'attribution', 'statEffect', '매력Effect', '지성Effect', '담력Effect', 
            '친절Effect', 'availableSale', 'dyeable', 'decomposable']
avatars = {
    'characterCode': None,
}
for part in avatar_parts:
    for key in dic_keys:
        avatars[f'{part}_{key}'] = None

In [95]:
sql = f"""
INSERT INTO Avatar (
    characterCode, 
    {''.join(f'{part}_{key}, ' for part in avatar_parts for key in dic_keys)[:-2]}
)
VALUES (%s, {''.join('%s, ' for _ in range(len(avatar_parts)*18))[:-2]});
"""

In [96]:
def return_tendency(text, tendency):
    try:
        # "tendency : " 다음의 숫자 위치를 찾습니다
        start_idx = text.find(f'{tendency} : ') + len(f'{tendency} : ')
        # 숫자가 시작하는 인덱스부터 숫자가 끝나는 인덱스까지 슬라이싱합니다
        number = text[start_idx:].split()[0]
        return int(number)
    except:
        return None

In [99]:
def set_avatar(fd, parts):
    avatar[f'{parts}_grade'] = fd['Grade']
    avatar[f'{parts}_isInner'] = fd['IsInner']
    avatar[f'{parts}_isSet'] = fd['IsSet']
    avatar[f'{parts}_name'] = fd['Name']
    avatar[f'{parts}_avatarType'] = fd['Type']
    
    for k, v in fd.items():
        sv = str(v)
        if (('희귀' in sv) | ('영웅' in sv) | ('전설' in sv)) & ('아바타' in sv): 
            avatar[f'{parts}_avatarType2'] = v
        elif ('전용' in sv) or ('계열' in sv):
            avatar[f'{parts}_availableClass'] = v
        elif '귀속' in sv:
            if (sv == '원정대 귀속됨 ') or (sv == '캐릭터 귀속됨'):
                avatar[f'{parts}_attribution'] = True
                avatar[f'{parts}_availableTrade'] = 0
            else:
                avatar[f'{parts}_attribution'] = False
                avatar[f'{parts}_availableTrade'] = int(int_pattern.findall(sv)[0])
        elif '+' in sv:
            avatar[f'{parts}_statEffect'] = float(float_pattern.search(sv).group())
        elif ('매력' in sv) or ('지성' in sv) or ('담력' in sv) or ('친절' in sv):
            if ('매력' in sv):
                avatar[f'{parts}_매력Effect'] = return_tendency(sv, '매력')
            if ('지성' in sv):
                avatar[f'{parts}_지성Effect'] = return_tendency(sv, '지성')
            if ('담력' in sv):
                avatar[f'{parts}_담력Effect'] = return_tendency(sv, '담력')
            if ('친절' in sv):
                avatar[f'{parts}_친절Effect'] = return_tendency(sv, '친절')
        elif ('판매' in sv) or ('분해' in sv) or ('염색' in sv):
            avatar[f'{parts}_availableSale'] = '판매불가' not in sv
            avatar[f'{parts}_dyeable'] = '염색불가' not in sv
            avatar[f'{parts}_decomposable'] = '분해불가' not in sv


In [100]:
# isInner가 True이다 == 2개 이상의 아바타를 끼고 있다? => 덧입기 중에는 효과가 발동되지 않음
for i in range(100):
    avatar = avatars.copy()
    avatar['characterCode'] = df['characterCode'][i]
    cdf = eval(df['data'][i])
    
    weapon_num = 0
    top_num = 0
    bottom_num = 0
    
    try:
        for j in range(len(cdf)):
            fd = flatten_dict(cdf[j])
            avatar_type = fd['Type']
            if avatar_type == '무기 아바타':
                if weapon_num == 0:
                    set_avatar(fd, '무기1')
                    weapon_num += 1
                else:
                    set_avatar(fd, '무기2')
            elif avatar_type == '상의 아바타':
                if top_num == 0:
                    set_avatar(fd, '상의1')
                    top_num += 1
                else:
                    set_avatar(fd, '상의2')
            elif avatar_type == '하의 아바타':
                if bottom_num == 0:
                    set_avatar(fd, '하의1')
                    bottom_num += 1
                else:
                    set_avatar(fd, '하의2')
            elif avatar_type =='머리 아바타':
                set_avatar(fd, '머리')
            elif avatar_type =='얼굴1 아바타':
                set_avatar(fd, '얼굴1')
            elif avatar_type =='얼굴2 아바타':
                set_avatar(fd, '얼굴2')
            elif avatar_type =='악기 아바타':
                set_avatar(fd, '악기')
            elif avatar_type == '':
                set_avatar(fd, '이동효과')
            
    except Exception as e:
        print(e)
        pass

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType'

# Armory Equipement

In [4]:
df = pd.read_csv(f'./data/ArmoryEquipment1.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [5]:
for i in range(2, 11):
    df2 = pd.read_csv(f'./data/ArmoryEquipment{i}.csv', usecols=[1, 2])
    df2 = df2.rename(columns={'0':'characterCode', '1':'data'})
    df = pd.concat([df, df2], axis=0)
df

,characterCode,data
0,0,"[{'Type': '무기', 'Name': '+7 분출된 격랑의 서브 머신건', '..."
1,1,None
2,2,"[{'Type': '무기', 'Name': '+8 눈부신 황혼의 붓', 'Icon'..."
3,3,"[{'Type': '무기', 'Name': '+18 타락한 마수의 사멸 검', 'I..."
4,4,None
...,...,...
9995,99997,"[{'Type': '무기', 'Name': '+19 광기의 갈망 리아네 하프', '..."
9996,99998,"[{'Type': '무기', 'Name': '인연의 리아네 하프', 'Icon': ..."
9997,99999,"[{'Type': '무기', 'Name': '+15 분출된 격랑의 헤비 건틀릿', ..."
9998,100000,None


In [6]:
df = df.reset_index(drop=True)

In [268]:
equipment_parts = ['무기', '투구', '상의', '하의', '어깨', '장갑']
accessory_parts = ['목걸이', '귀걸이1', '귀걸이2', '반지1', '반지2'] # + 팔찌, 어빌리티 스톤
sequipment_parts = ['나침반','부적','휘장']
# 무기 장비
aedic_keys = ['grade', 'name', 'type', 'quality', 'tier', 'itemLevel', 'ATK', 'AD(%)', 
              'setName', 'setLevel', 'REXP', 'esther', 'estherEffect','ella', 'reinforcementStep']
# 방어구 장비
dedic_keys = ['grade', 'name', 'type', 'quality', 'tier', 'itemLevel', 
              'SAI', 'BDEF', 'BMDEF', 'BHP', 'ADEF', 'AMDEF', 'AHP', 'AHPP',
              'setName', 'setLevel', 'REXP', 'reinforcementStep',
             'alchemyName1', 'alchemyName2', 'alchemyPoint1', 'alchemyPoint2']
# 악세서리 장비
adic_keys = ['grade', 'name', 'quality', 'tier', 'limitLevel', 'availableTrade', 
             'SAI', 'HP', '치명', '특화', '신속', '제압', '인내', '숙련',
             'engraving1_name', 'engraving1_point', 'engraving2_name', 'engraving2_point', 
             'dengraving_name', 'dengraving_point', 'acquirablePlace']

# 장비
equipments = {
    'characterCode': None,
}
# 악세서리
accessories = {
    'characterCode': None,
    '팔찌_grade':None, '팔찌_name':None, '팔찌_tier':None, '팔찌_effect1':None, '팔찌_effect2':None,
    '팔찌_effect3':None, '팔찌_effect4':None, '팔찌_effect5':None, '팔찌_canRerollNum':None, '팔찌_canGivenNum':None,
    'AS_grade':None, 'AS_name':None, 'AS_tier':None, 'AS_HP': None, 'AS_BHP':None, 'AS_engraving1_name':None, 
    'AS_engraving1_point':None, 'AS_engraving2_name':None, 'AS_engraving2_point':None,
    'AS_dengraving_name':None, 'AS_dengraving_point':None, 'AS_setLevel' :None
}
# 특수 장비
sequipments = {
    'characterCode':None,
}
for part in equipment_parts:
    if part == '무기':
        for key in aedic_keys:
            equipments[f'{part}_{key}'] = None
    else:
        for key in dedic_keys:
            equipments[f'{part}_{key}'] = None
for part in accessory_parts:
    for key in adic_keys:
        accessories[f'{part}_{key}'] = None

In [269]:
sql = """
INSERT INTO equipment_table ('characterCode', '무기_grade', '무기_name', '무기_type', '무기_quality', '무기_tier', '무기_itemLevel', '무기_ATK', `무기_AD(%%)`, '무기_setName', '무기_setLevel', '무기_REXP', '무기_esther', '무기_estherEffect', '무기_ella', '무기_reinforcementStep', '투구_grade', '투구_name', '투구_type', '투구_quality', '투구_tier', '투구_itemLevel', '투구_SAI', '투구_BDEF', '투구_BMDEF', '투구_BHP', '투구_ADEF', '투구_AMDEF', '투구_AHP', '투구_AHPP', '투구_setName', '투구_setLevel', '투구_REXP', '투구_reinforcementStep', '투구_alchemyName1', '투구_alchemyName2', '투구_alchemyPoint1', '투구_alchemyPoint2', '상의_grade', '상의_name', '상의_type', '상의_quality', '상의_tier', '상의_itemLevel', '상의_SAI', '상의_BDEF', '상의_BMDEF', '상의_BHP', '상의_ADEF', '상의_AMDEF', '상의_AHP', '상의_AHPP', '상의_setName', '상의_setLevel', '상의_REXP', '상의_reinforcementStep', '상의_alchemyName1', '상의_alchemyName2', '상의_alchemyPoint1', '상의_alchemyPoint2', '하의_grade', '하의_name', '하의_type', '하의_quality', '하의_tier', '하의_itemLevel', '하의_SAI', '하의_BDEF', '하의_BMDEF', '하의_BHP', '하의_ADEF', '하의_AMDEF', '하의_AHP', '하의_AHPP', '하의_setName', '하의_setLevel', '하의_REXP', '하의_reinforcementStep', '하의_alchemyName1', '하의_alchemyName2', '하의_alchemyPoint1', '하의_alchemyPoint2', '어깨_grade', '어깨_name', '어깨_type', '어깨_quality', '어깨_tier', '어깨_itemLevel', '어깨_SAI', '어깨_BDEF', '어깨_BMDEF', '어깨_BHP', '어깨_ADEF', '어깨_AMDEF', '어깨_AHP', '어깨_AHPP', '어깨_setName', '어깨_setLevel', '어깨_REXP', '어깨_reinforcementStep', '어깨_alchemyName1', '어깨_alchemyName2', '어깨_alchemyPoint1', '어깨_alchemyPoint2', '장갑_grade', '장갑_name', '장갑_type', '장갑_quality', '장갑_tier', '장갑_itemLevel', '장갑_SAI', '장갑_BDEF', '장갑_BMDEF', '장갑_BHP', '장갑_ADEF', '장갑_AMDEF', '장갑_AHP', '장갑_AHPP', '장갑_setName', '장갑_setLevel', '장갑_REXP', '장갑_reinforcementStep', '장갑_alchemyName1', '장갑_alchemyName2', '장갑_alchemyPoint1', '장갑_alchemyPoint2')
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [271]:
plus_pattern = re.compile(r'([^\s]+)\s*\+([\d]+)')
bracelet_pattern = re.compile(r'\[.*?\[')
for i in range(10000):
    equipment = equipments.copy()
    accessory = accessories.copy()
    sequipment = sequipments.copy()
    equipment['characterCode'] = df['characterCode'][i]
    accessory['characterCode'] = df['characterCode'][i]
    sequipment['characterCode'] = df['characterCode'][i]
    cdf = eval(df['data'][i])
    
    earring_count = 0
    ring_count = 0
    
    try:
        for j in range(len(cdf)):
            fd = flatten_dict(cdf[j])
            if '무기' == fd['Type']:
                equipment['무기_grade'] = fd['Grade']
                equipment['무기_name'] = fd['Name'].split(' ', 1)[1]
                if '+' in fd['Name']:
                    equipment['무기_reinforcementStep'] = fd['Name'].split(' ', 1)[0].replace('+', '')
                else:
                    equipment['무기_reinforcementStep'] = 0
                for k, v in fd.items():
                    sv = str(v)
                    sk = str(k)
                    if ('아이템 레벨' in sv) & ('티어' in sv):
                        equipment['무기_itemLevel'] = int(int_pattern.findall(sv.split(' (')[0])[0])
                        equipment['무기_tier'] = int(int_pattern.findall(sv.split(' (')[1])[0])
                    elif ('무기 공격력 +' in sv) & ('이동 속도' not in sv) :
                        equipment['무기_ATK'] = int(int_pattern.findall(sv)[0])
                    elif '추가 피해 +' in sv:
                        equipment['무기_AD(%)'] = float(float_pattern.search(sv).group())
                    elif 'qualityValue' in sk:
                        equipment['무기_quality'] = v
                    elif 'Tooltip_Element_001_value_leftStr0' in sk:
                        equipment['무기_type'] = sv
                    elif ('Tooltip_Element_008_value_Element_001' in sk) & (equipment['무기_setName'] == None):
                        equipment['무기_setName'] = sv.split(' Lv.')[0]
                        equipment['무기_setLevel'] = int(sv.split(' Lv.')[1])
                    elif 'maximum' in sk:
                        equipment['무기_REXP'] = v
                    elif '에스더 효과' in sv:
                        equipment['무기_estherEffect'] = sv.split('[')[1].replace(']', '')
                    elif '엘라 부여 완료' in sv:
                        equipment['무기_ella'] = True
                    elif sv == '에스더': # 에스더 세트 네임, 레벨 
                        equipment['무기_esther'] = True
                        equipment['무기_ella'] = False # 항상 sv==에스더가 먼저옴 -> 우선 False로 설정
                        cvalues = eval(df[df['characterCode'] == df['characterCode'][i]]['data'].values[0])
                        cfd = flatten_dict(cvalues[1])
                        for ck, cv in cfd.items():
                            scv = str(cv)
                            if '장갑' in scv:
                                set_name = scv.split(' (')[0]
                                set_level = int(int_pattern.findall(scv)[0])
                                equipment['무기_setName'] = set_name
                                equipment['무기_setLevel'] = set_level
                                break

            elif fd['Type'] in ['투구', '어깨', '상의', '하의', '장갑']:
                archemy_count = 0
                etype = fd['Type']
                equipment[f'{etype}_grade'] = fd['Grade']
                equipment[f'{etype}_name'] = fd['Name'].split(' ', 1)[1]
                if '+' in fd['Name']:
                    equipment[f'{etype}_reinforcementStep'] = fd['Name'].split(' ', 1)[0].replace('+', '')
                else:
                    equipment[f'{etype}_reinforcementStep'] = 0

                for k, v in fd.items():
                    sv = str(v)
                    sk = str(k)
                    if ('아이템 레벨' in sv) & ('티어' in sv):
                        equipment[f'{etype}_itemLevel'] = int(int_pattern.findall(sv.split(' (')[0])[0])
                        equipment[f'{etype}_tier'] = int(int_pattern.findall(sv.split(' (')[1])[0])
                    elif (('힘 +' in sv) | ('민첩 +' in sv) | ('지능 +' in sv)) & ('방어력' in sv):
                        res = int_pattern.findall(sv)
                        equipment[f'{etype}_BDEF'] = int(res[0])
                        equipment[f'{etype}_BMDEF'] = int(res[1])
                        equipment[f'{etype}_SAI'] = int(res[2])
                        equipment[f'{etype}_BHP'] = int(res[3])
                    elif (('힘 +' not in sv) | ('민첩 +' not in sv) | ('지능 +' not in sv))\
                    & (('방어력' in sv) | ('생명 활성력' in sv)):
                        res = int_pattern.findall(sv)
                        if len(res)>1:
                            equipment[f'{etype}_ADEF'] = int(res[0])
                            equipment[f'{etype}_AMDEF'] = int(res[1])
                            equipment[f'{etype}_AHP'] = int(res[2])
                        else:
                            equipment[f'{etype}_AHPP'] = int(res[0])
                    elif 'maximum' in sk:
                        equipment[f'{etype}_REXP'] = v
                    elif 'qualityValue' in sk:
                        equipment[f'{etype}_quality'] = v
                    elif 'Tooltip_Element_001_value_leftStr0' in sk:
                        equipment[f'{etype}_type'] = sv
                    elif ('Tooltip_Element_008_value_Element_001' in sk) & (equipment[f'{etype}_setName'] == None):
                        equipment[f'{etype}_setName'] = sv.split(' Lv.')[0]
                        equipment[f'{etype}_setLevel'] = int(sv.split(' Lv.')[1])
                    elif (('공용' in sv) | ('질서' in sv) | ('혼돈' in sv) | \
                          ('[견갑]' in sv) | ('[상의]' in sv) | ('[하의]' in sv)) \
                    & ('혼돈 강화' not in  sv):
                        archemy_count += 1
                        archemy = archemy_pattern.findall(sv)[0]
                        equipment[f'{etype}_alchemyName{archemy_count}'] = archemy[0].strip()
                        equipment[f'{etype}_alchemyLevel{archemy_count}'] = int(archemy[1])
            elif fd['Type'] in ['목걸이', '귀걸이', '반지']:
                engraving_count = 0
                atype = fd['Type']
                if atype == '귀걸이': 
                    earring_count += 1
                    atype += str(earring_count)
                elif atype == '반지': 
                    ring_count += 1
                    atype += str(ring_count)
                accessory[f'{atype}_grade'] = fd['Grade']
                accessory[f'{atype}_name'] = fd['Name']
                for k, v in fd.items():
                    sk = str(k)
                    sv = str(v)
                    if 'qualityValue' in sv:
                        accessory[f'{atype}_quality'] = v 
                    elif '티어' in sv:
                        accessory[f'{atype}_tier'] = int(int_pattern.findall(sv)[0])
                    elif '아이템 레벨' in sv:
                        ssv = sv.split('아이템 레벨')
                        accessory[f'{atype}_limitLevel'] = int(ssv[1])
                        try:
                            res = int_pattern.findall(ssv[0])[0]
                            accessory[f'{atype}_availableTrade'] = int(res)
                        except:
                            accessory[f'{atype}_availableTrade'] = 0
                    elif '체력' in sv:
                        res = int_pattern.findall(sv)
                        accessory[f'{atype}_SAI'] = int(res[0])
                        accessory[f'{atype}_HP'] = int(res[-1])
                    elif (('반지' not in sv) & ('귀걸이' not in sv) & ('목걸이' not in sv)) & \
                    (('치명' in sv) | ('특화' in sv) | ('신속' in sv) | ('제압' in sv) | ('인내' in sv) \
                     | ('숙련' in sv)):
                        svrs = sv.replace('+', '').split(' ')
                        for idx in range(0, len(svrs), 2):
                            accessory[f'{atype}_{svrs[idx]}'] = int(svrs[idx+1])
                    elif '활성도' in sv:
                        svrs = sv.replace('[', '').replace(']', '').replace('+', '').split(' 활성도')
                        if '감소' in svrs[0]:
                            accessory[f'{atype}_dengraving_name'] = svrs[0]
                            accessory[f'{atype}_dengraving_point'] = int(svrs[1])
                        else:
                            engraving_count += 1
                            accessory[f'{atype}_engraving{engraving_count}_name'] = svrs[0]
                            accessory[f'{atype}_engraving{engraving_count}_point'] = int(svrs[1])
                    elif ('군단장' in sv) | ('어비스' in sv) | ('카오스 던전' in sv) | ('필드 보스' in sv) |\
                    ('가디언 토벌' in sv):
                        accessory[f'{atype}_acquirablePlace'] = sv
            elif fd['Type'] =='팔찌':
                accessory['팔찌_grade'] = fd['Grade']
                accessory['팔찌_name'] = fd['Name']
                for k, v in fd.items():
                    sk = str(k)
                    sv = str(v)
                    if '티어' in sv:
                        accessory['팔찌_tier'] = int(int_pattern.findall(sv)[0])
                    elif '효과 부여' in sv:
                        try:
                            svn = int(int_pattern.findall(sv)[0])
                            accessory['팔찌_canRerollNum'] = svn
                        except:
                            accessory['팔찌_canRerollNum'] = 0
                    elif 'Tooltip_Element_004_value_Element_001' in sk:
                        effect_count = 1
                        canGivenNum = 1
                        sv= sv.replace(')', ') ')
                        while '효과 부여 가능' in sv:
                            accessory['팔찌_canGivenNum'] = canGivenNum
                            canGivenNum += 1
                            sv = sv.replace('효과 부여 가능', '', 1)
                        if canGivenNum == 1:
                            accessory['팔찌_canGivenNum'] = 0
                        for v in plus_pattern.findall(sv):
                            v = list(v)
                            v[0] = v[0].replace('회복량', '전투 중 생명력 회복량').replace('마나', '최대 마나')\
                            .replace('공격력', '무기 공격력').replace('생명력', '최대 생명력')
                            if v[0] == '방어력':
                                svidx = sv.find(f'{v[0]} +{v[1]}')
                                v[0] = v[0].replace('방어력', f'{sv[svidx-3:svidx]}방어력')
                            accessory[f'팔찌_effect{effect_count}'] = v
                            effect_count += 1
                            sv = sv.replace(f'{v[0]} +{v[1]}', '')
                        while len(bracelet_pattern.findall(sv)) != 0:
                            accessory[f'팔찌_effect{effect_count}'] = bracelet_pattern.findall(sv)[0][:-2].strip()
                            sv = sv.replace(bracelet_pattern.findall(sv)[0][:-2], '')
                            effect_count += 1
                        if len(sv) > 1:
                            accessory[f'팔찌_effect{effect_count}'] = sv.strip()
            elif fd['Type'] == '어빌리티 스톤':
                engraving_count = 0 
                accessory['AS_grade'] = fd['Grade']
                accessory['AS_name'] = fd['Name']
                if 'IV' in fd['Name']:
                    accessory['AS_setLevel'] = 4
                elif 'III' in fd['Name']:
                    accessory['AS_setLevel'] = 3
                elif 'II' in fd['Name']:
                    accessory['AS_setLevel'] = 2
                elif 'I' in fd['Name']:
                    accessory['AS_setLevel'] = 1
                else:
                    accessory['AS_setLevel'] = 0
                accessory['AS_BHP'] = 0
                for k, v in fd.items():
                    sv = str(v)
                    sk = str(k)
                    if '티어' in sv:
                        accessory['AS_tier'] = int(sv[-1])
                    elif sk == 'Tooltip_Element_004_value_Element_001':
                        accessory['AS_HP'] = int(int_pattern.findall(sv)[0])
                    elif sk == 'Tooltip_Element_005_value_Element_001':
                        accessory['AS_BHP'] = int(int_pattern.findall(sv)[0])
                    elif '활성도' in sv:
                        svrs = sv.replace('[', '').replace(']', '').replace('+', '').split(' 활성도')
                        if '감소' in svrs[0]:
                            accessory[f'AS_dengraving_name'] = svrs[0]
                            accessory[f'AS_dengraving_point'] = int(svrs[1])
                        else:
                            engraving_count += 1
                            accessory[f'AS_engraving{engraving_count}_name'] = svrs[0]
                            accessory[f'AS_engraving{engraving_count}_point'] = int(svrs[1])
                        
        pprint(accessory)
    except Exception as e:
#         pass
        print(e)

{'AS_BHP': 0,
 'AS_HP': 2554,
 'AS_dengraving_name': '공격력 감소',
 'AS_dengraving_point': 5,
 'AS_engraving1_name': '원한',
 'AS_engraving1_point': 7,
 'AS_engraving2_name': '저주받은 인형',
 'AS_engraving2_point': 5,
 'AS_grade': '유물',
 'AS_name': '고고한 정진의 돌',
 'AS_setLevel': 0,
 'AS_tier': 2,
 'characterCode': 0,
 '귀걸이1_HP': 1659,
 '귀걸이1_SAI': 5578,
 '귀걸이1_acquirablePlace': '[어비스 레이드] 아르고스[어비스 던전] 오레하의 우물',
 '귀걸이1_availableTrade': 3,
 '귀걸이1_dengraving_name': '공격력 감소',
 '귀걸이1_dengraving_point': 1,
 '귀걸이1_engraving1_name': '에테르 포식자',
 '귀걸이1_engraving1_point': 2,
 '귀걸이1_engraving2_name': '저주받은 인형',
 '귀걸이1_engraving2_point': 2,
 '귀걸이1_grade': '전설',
 '귀걸이1_limitLevel': 1100,
 '귀걸이1_name': '추락한 영겁의 귀걸이',
 '귀걸이1_quality': None,
 '귀걸이1_tier': 3,
 '귀걸이1_숙련': None,
 '귀걸이1_신속': None,
 '귀걸이1_인내': None,
 '귀걸이1_제압': None,
 '귀걸이1_치명': 270,
 '귀걸이1_특화': None,
 '귀걸이2_HP': 1565,
 '귀걸이2_SAI': 5097,
 '귀걸이2_acquirablePlace': None,
 '귀걸이2_availableTrade': 2,
 '귀걸이2_dengraving_name': '공격속도 감소',
 '귀걸이2_dengraving_point

{'AS_BHP': 1526,
 'AS_HP': 15254,
 'AS_dengraving_name': '공격속도 감소',
 'AS_dengraving_point': 4,
 'AS_engraving1_name': '저주받은 인형',
 'AS_engraving1_point': 8,
 'AS_engraving2_name': '기습의 대가',
 'AS_engraving2_point': 7,
 'AS_grade': '유물',
 'AS_name': '고고한 비상의 돌',
 'AS_setLevel': 0,
 'AS_tier': 3,
 'characterCode': 7,
 '귀걸이1_HP': 2034,
 '귀걸이1_SAI': 7678,
 '귀걸이1_acquirablePlace': '[가디언 토벌] 데스칼루다[가디언 토벌] 쿤겔라니움',
 '귀걸이1_availableTrade': 2,
 '귀걸이1_dengraving_name': '이동속도 감소',
 '귀걸이1_dengraving_point': 2,
 '귀걸이1_engraving1_name': '일격필살',
 '귀걸이1_engraving1_point': 3,
 '귀걸이1_engraving2_name': '저주받은 인형',
 '귀걸이1_engraving2_point': 4,
 '귀걸이1_grade': '유물',
 '귀걸이1_limitLevel': 1415,
 '귀걸이1_name': '찬란한 파멸자의 귀걸이',
 '귀걸이1_quality': None,
 '귀걸이1_tier': 3,
 '귀걸이1_숙련': None,
 '귀걸이1_신속': None,
 '귀걸이1_인내': None,
 '귀걸이1_제압': None,
 '귀걸이1_치명': None,
 '귀걸이1_특화': 272,
 '귀걸이2_HP': 2034,
 '귀걸이2_SAI': 7678,
 '귀걸이2_acquirablePlace': '[필드 보스] 로웬 - 헤르무트[카오스 던전] 베른 남부',
 '귀걸이2_availableTrade': 2,
 '귀걸이2_dengraving_name':

 '목걸이_engraving1_point': 2,
 '목걸이_engraving2_name': '슈퍼 차지',
 '목걸이_engraving2_point': 3,
 '목걸이_grade': '전설',
 '목걸이_limitLevel': 1100,
 '목걸이_name': '빛나는 구도자의 목걸이',
 '목걸이_quality': None,
 '목걸이_tier': 3,
 '목걸이_숙련': None,
 '목걸이_신속': 449,
 '목걸이_인내': None,
 '목걸이_제압': None,
 '목걸이_치명': 433,
 '목걸이_특화': None,
 '반지1_HP': 1327,
 '반지1_SAI': 5179,
 '반지1_acquirablePlace': '[가디언 토벌] 토벌 5단계',
 '반지1_availableTrade': 3,
 '반지1_dengraving_name': '이동속도 감소',
 '반지1_dengraving_point': 2,
 '반지1_engraving1_name': '구슬동자',
 '반지1_engraving1_point': 3,
 '반지1_engraving2_name': '마나의 흐름',
 '반지1_engraving2_point': 2,
 '반지1_grade': '전설',
 '반지1_limitLevel': 1100,
 '반지1_name': '빛나는 파멸자의 반지',
 '반지1_quality': None,
 '반지1_tier': 3,
 '반지1_숙련': None,
 '반지1_신속': 159,
 '반지1_인내': None,
 '반지1_제압': None,
 '반지1_치명': None,
 '반지1_특화': None,
 '반지2_HP': 1327,
 '반지2_SAI': 5179,
 '반지2_acquirablePlace': '[가디언 토벌] 토벌 5단계',
 '반지2_availableTrade': 3,
 '반지2_dengraving_name': '공격속도 감소',
 '반지2_dengraving_point': 1,
 '반지2_engraving1_name': '예리한 둔기

 '목걸이_engraving2_name': '아드레날린',
 '목걸이_engraving2_point': 3,
 '목걸이_grade': '유물',
 '목걸이_limitLevel': 1415,
 '목걸이_name': '울부짖는 혼돈의 목걸이',
 '목걸이_quality': None,
 '목걸이_tier': 3,
 '목걸이_숙련': None,
 '목걸이_신속': 493,
 '목걸이_인내': None,
 '목걸이_제압': None,
 '목걸이_치명': 497,
 '목걸이_특화': None,
 '반지1_HP': 1628,
 '반지1_SAI': 7130,
 '반지1_acquirablePlace': '[군단장 레이드] 마수군단장 발탄[군단장 레이드] 욕망군단장 비아키스[군단장 레이드] 광기군단장 '
                        '쿠크세이튼',
 '반지1_availableTrade': 1,
 '반지1_dengraving_name': '이동속도 감소',
 '반지1_dengraving_point': 2,
 '반지1_engraving1_name': '돌격대장',
 '반지1_engraving1_point': 4,
 '반지1_engraving2_name': '원한',
 '반지1_engraving2_point': 3,
 '반지1_grade': '유물',
 '반지1_limitLevel': 1415,
 '반지1_name': '울부짖는 혼돈의 반지',
 '반지1_quality': None,
 '반지1_tier': 3,
 '반지1_숙련': None,
 '반지1_신속': 200,
 '반지1_인내': None,
 '반지1_제압': None,
 '반지1_치명': None,
 '반지1_특화': None,
 '반지2_HP': 1628,
 '반지2_SAI': 7130,
 '반지2_acquirablePlace': '[군단장 레이드] 마수군단장 발탄[군단장 레이드] 욕망군단장 비아키스[군단장 레이드] 광기군단장 '
                        '쿠크세이튼',
 '반지2_avai

 '목걸이_tier': 3,
 '목걸이_숙련': None,
 '목걸이_신속': 464,
 '목걸이_인내': None,
 '목걸이_제압': None,
 '목걸이_치명': None,
 '목걸이_특화': 499,
 '반지1_HP': 1890,
 '반지1_SAI': 9061,
 '반지1_acquirablePlace': '[군단장 레이드] 질병군단장 일리아칸[어비스 던전] 카양겔[어비스 던전] 혼돈의 상아탑',
 '반지1_availableTrade': 2,
 '반지1_dengraving_name': '방어력 감소',
 '반지1_dengraving_point': 3,
 '반지1_engraving1_name': '절실한 구원',
 '반지1_engraving1_point': 6,
 '반지1_engraving2_name': '각성',
 '반지1_engraving2_point': 3,
 '반지1_grade': '고대',
 '반지1_limitLevel': 1540,
 '반지1_name': '참혹한 몰락의 반지',
 '반지1_quality': None,
 '반지1_tier': 3,
 '반지1_숙련': None,
 '반지1_신속': 196,
 '반지1_인내': None,
 '반지1_제압': None,
 '반지1_치명': None,
 '반지1_특화': None,
 '반지2_HP': 1890,
 '반지2_SAI': 9061,
 '반지2_acquirablePlace': '[카오스 던전] 엘가시아[카오스 던전] 볼다이크[필드 보스] 볼다이크 - 드라커스',
 '반지2_availableTrade': 2,
 '반지2_dengraving_name': '공격속도 감소',
 '반지2_dengraving_point': 3,
 '반지2_engraving1_name': '절실한 구원',
 '반지2_engraving1_point': 6,
 '반지2_engraving2_name': '전문의',
 '반지2_engraving2_point': 3,
 '반지2_grade': '고대',
 '반지2_limitLevel

...

 '귀걸이2_치명': None,
 '귀걸이2_특화': None,
 '목걸이_HP': 1820,
 '목걸이_SAI': 4951,
 '목걸이_acquirablePlace': None,
 '목걸이_availableTrade': None,
 '목걸이_dengraving_name': None,
 '목걸이_dengraving_point': None,
 '목걸이_engraving1_name': '중갑 착용',
 '목걸이_engraving1_point': 4,
 '목걸이_engraving2_name': None,
 '목걸이_engraving2_point': None,
 '목걸이_grade': '유물',
 '목걸이_limitLevel': None,
 '목걸이_name': '위대한 모험의 목걸이',
 '목걸이_quality': None,
 '목걸이_tier': 2,
 '목걸이_숙련': None,
 '목걸이_신속': 350,
 '목걸이_인내': 350,
 '목걸이_제압': None,
 '목걸이_치명': 350,
 '목걸이_특화': 350,
 '반지1_HP': 1040,
 '반지1_SAI': 3576,
 '반지1_acquirablePlace': None,
 '반지1_availableTrade': None,
 '반지1_dengraving_name': None,
 '반지1_dengraving_point': None,
 '반지1_engraving1_name': '중갑 착용',
 '반지1_engraving1_point': 3,
 '반지1_engraving2_name': None,
 '반지1_engraving2_point': None,
 '반지1_grade': '유물',
 '반지1_limitLevel': None,
 '반지1_name': '위대한 모험의 반지',
 '반지1_quality': None,
 '반지1_tier': 2,
 '반지1_숙련': None,
 '반지1_신속': 150,
 '반지1_인내': None,
 '반지1_제압': 150,
 '반지1_치명': None,
 '반지1_특화

 '반지2_availableTrade': 2,
 '반지2_dengraving_name': '공격속도 감소',
 '반지2_dengraving_point': 1,
 '반지2_engraving1_name': '타격의 대가',
 '반지2_engraving1_point': 6,
 '반지2_engraving2_name': '아드레날린',
 '반지2_engraving2_point': 3,
 '반지2_grade': '고대',
 '반지2_limitLevel': 1540,
 '반지2_name': '공허한 인연의 반지',
 '반지2_quality': None,
 '반지2_tier': 3,
 '반지2_숙련': None,
 '반지2_신속': None,
 '반지2_인내': None,
 '반지2_제압': None,
 '반지2_치명': 190,
 '반지2_특화': None,
 '팔찌_canGivenNum': 0,
 '팔찌_canRerollNum': 0,
 '팔찌_effect1': ['체력', '2125'],
 '팔찌_effect2': ['민첩', '2676'],
 '팔찌_effect3': '[우월] 몬스터에게 공격 적중 시 주는 피해가 2.5%증가한다. (60레벨 초과 몬스터에게는 효과 감소)',
 '팔찌_effect4': "[망치] 몬스터에게 공격 적중 시 8초 동안 '망치' 효과를 획득한다. '강철 쐐기' 효과를 보유 시 치명타 "
               '피해가 8% 추가 증가한다. 망치 : 공격 적중 시 치명타 피해량이 10%증가한다. (60레벨 초과 몬스터에게는 '
               '효과 감소)',
 '팔찌_effect5': "[열정] 자신의 생명력이 40% 이상일 경우 적에게 공격 적중 시 3초 동안 '열정' 효과를 획득한다. '냉정' "
               "효과를 보유 중 일 때 '열정' 효과가 1% 추가 증가한다. 열정 : 몬스터에게 주는 피해가 3%증가한다. "
               '(60레벨 초과 몬스터에게는 효과 감소)',
 '팔찌_gra

 '귀걸이1_dengraving_name': None,
 '귀걸이1_dengraving_point': None,
 '귀걸이1_engraving1_name': None,
 '귀걸이1_engraving1_point': None,
 '귀걸이1_engraving2_name': None,
 '귀걸이1_engraving2_point': None,
 '귀걸이1_grade': None,
 '귀걸이1_limitLevel': None,
 '귀걸이1_name': None,
 '귀걸이1_quality': None,
 '귀걸이1_tier': None,
 '귀걸이1_숙련': None,
 '귀걸이1_신속': None,
 '귀걸이1_인내': None,
 '귀걸이1_제압': None,
 '귀걸이1_치명': None,
 '귀걸이1_특화': None,
 '귀걸이2_HP': None,
 '귀걸이2_SAI': None,
 '귀걸이2_acquirablePlace': None,
 '귀걸이2_availableTrade': None,
 '귀걸이2_dengraving_name': None,
 '귀걸이2_dengraving_point': None,
 '귀걸이2_engraving1_name': None,
 '귀걸이2_engraving1_point': None,
 '귀걸이2_engraving2_name': None,
 '귀걸이2_engraving2_point': None,
 '귀걸이2_grade': None,
 '귀걸이2_limitLevel': None,
 '귀걸이2_name': None,
 '귀걸이2_quality': None,
 '귀걸이2_tier': None,
 '귀걸이2_숙련': None,
 '귀걸이2_신속': None,
 '귀걸이2_인내': None,
 '귀걸이2_제압': None,
 '귀걸이2_치명': None,
 '귀걸이2_특화': None,
 '목걸이_HP': None,
 '목걸이_SAI': None,
 '목걸이_acquirablePlace': None,
 '목걸이_availableTrade': 

 '귀걸이2_인내': None,
 '귀걸이2_제압': None,
 '귀걸이2_치명': None,
 '귀걸이2_특화': 290,
 '목걸이_HP': 3308,
 '목걸이_SAI': 12546,
 '목걸이_acquirablePlace': '[가디언 토벌] 소나벨[가디언 토벌] 가르가디스',
 '목걸이_availableTrade': 2,
 '목걸이_dengraving_name': '공격속도 감소',
 '목걸이_dengraving_point': 2,
 '목걸이_engraving1_name': '예리한 둔기',
 '목걸이_engraving1_point': 5,
 '목걸이_engraving2_name': '원한',
 '목걸이_engraving2_point': 3,
 '목걸이_grade': '고대',
 '목걸이_limitLevel': 1540,
 '목걸이_name': '거룩한 선지자의 목걸이',
 '목걸이_quality': None,
 '목걸이_tier': 3,
 '목걸이_숙련': None,
 '목걸이_신속': 461,
 '목걸이_인내': None,
 '목걸이_제압': None,
 '목걸이_치명': None,
 '목걸이_특화': 500,
 '반지1_HP': 1890,
 '반지1_SAI': 9061,
 '반지1_acquirablePlace': '[군단장 레이드] 몽환군단장 아브렐슈드 - 하드[어비스 던전] 카양겔',
 '반지1_availableTrade': 1,
 '반지1_dengraving_name': '공격력 감소',
 '반지1_dengraving_point': 3,
 '반지1_engraving1_name': '원한',
 '반지1_engraving1_point': 6,
 '반지1_engraving2_name': '저주받은 인형',
 '반지1_engraving2_point': 3,
 '반지1_grade': '고대',
 '반지1_limitLevel': 1540,
 '반지1_name': '참혹한 파멸의 반지',
 '반지1_quality': None,
 '반지1_tier': 3

KeyboardInterrupt: 

# ArmorySkills

In [102]:
df = pd.read_csv('./data/ArmorySkills.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [28]:
skill_parts = [f'skill{i}' for i in range(1,17)] 
dic_keys = ['name', 'level', 'cooltime','tripod1_name', 'tripod1_point', 'tripod2_name', 'tripod2_point',
            'tripod3_name', 'tripod3_point', 'runeName', 'runeGrade','headAttack', 'backAttack', 
            'partBreak', 'stagger', 'counter', '마나', '배터리', '버블', '충격', '기공', '기력', '내공', 
            '경직면역', '피격면역', '상태이상면역']
skills = {
    'characterCode': None,
}
for part in skill_parts:
    for key in dic_keys:
        skills[f'{part}_{key}'] = None

['skill1',
 'skill2',
 'skill3',
 'skill4',
 'skill5',
 'skill6',
 'skill7',
 'skill8',
 'skill9',
 'skill10',
 'skill11',
 'skill12',
 'skill13',
 'skill14',
 'skill15',
 'skill16']

In [104]:
for i in range(100):
    skill = skills.copy()
    skill['characterCode'] = df['characterCode'][i]
    cdf = eval(df['data'][i])
    skill_count = 0
    try:
        for j in range(len(cdf)):
            fd = flatten_dict(cdf[j])
            try: 
                # 룬을 끼고 있지 않다면
                fd['Rune']
                exist_rune = False
            except: 
                exist_rune=True
            
            if (fd['Level'] > 1) or (exist_rune == True):
                skill_count += 1
                tripod_count = 0
                skill[f'skill{skill_count}_level'] = fd['Level']
                skill[f'skill{skill_count}_name'] = fd['Name']
                if exist_rune == True:
                    skill[f'skill{skill_count}_runeName'] = fd['Rune_Name']
                    skill[f'skill{skill_count}_runeGrade'] = fd['Rune_Grade']
#                 pprint(fd)
                for k, v in fd.items():
                    sv = str(v)
                    sk = str(k)
                    if ('_value_leftText' in sk) & ('재사용 대기시간' in sv):
                        skill[f'skill{skill_count}_cooltime'] = int_pattern.findall(sv)[0]
                    elif ('Tooltip_Element_' in sk)  &  ('소모' in sv) & ('않는다' not in sv) & \
                    ('증가' not in sv) & ('감소' not in sv) & ('회복' not in sv):
                        sv = sv.replace('|', '')
                        sv = sv.split(', ')
                        resourceName = resource_pattern.findall(sv[0])[0]
                        if len(sv) > 1: # 인파
                            resourceName2 = resource_pattern.findall(sv[1])[0]
                            if '모두 소모' in sv[0]:
                                skill[f'skill{skill_count}_{resourceName}'] = -100
                            skill[f'skill{skill_count}_{resourceName}'] = -1 * int(int_pattern.findall(sv[0])[0])
                            skill[f'skill{skill_count}_{resourceName2}'] = int(int_pattern.findall(sv[1])[0])
                        else:
                            skill[f'skill{skill_count}_{resourceName}'] = -1 * int(int_pattern.findall(sv[0])[0])
                    elif ('_value_Element_' in sk) & ('name' in sk) & (sv != ''):
                        tripod_count+=1
                        skill[f'skill{skill_count}_tripod{tripod_count}_name'] = sv
                    elif ('_value_Element_' in sk) & ('tier' in sk) & (sv != ''):
                        skill[f'skill{skill_count}_tripod{tripod_count}_point'] = int(int_pattern.findall(sv)[0])
                    elif ('공격 타입 :' in sv) | ('부위 파괴 :' in sv) | ('무력화 :' in sv) | ('슈퍼아머 :' in sv) |\
                    ('카운터 :' in sv):
                        if ('공격 타입 :' in sv) :
                            skill[f'skill{skill_count}_headAttack'] = '헤드 어택' in sv
                            skill[f'skill{skill_count}_backAttack'] = '백 어택' in sv
                        if ('부위 파괴 :' in sv) :
                            start_idx = sv.find('부위 파괴 : 레벨') + len('부위 파괴 : 레벨')
                            number = sv[start_idx:].split()[0].replace('무력화', '').replace('공격', '')\
                            .replace('슈퍼아머', '')
                            skill[f'skill{skill_count}_partBreak'] = int(number)
                        else:
                            skill[f'skill{skill_count}_partBreak'] = 0
                        if ('무력화 :' in sv) :
                            start_idx = sv.find('무력화 : ') + len('무력화 : ')
                            stagger = sv[start_idx:].split()[0].replace('공격', '').replace('카운터', '')\
                            .replace('슈퍼아머', '')
                            skill[f'skill{skill_count}_stagger'] = stagger
                        else: 
                            skill[f'skill{skill_count}_stagger'] = 0
                        skill[f'skill{skill_count}_counter'] = '카운터' in sv
                        skill[f'skill{skill_count}_경직면역'] = '경직 면역' in sv
                        skill[f'skill{skill_count}_피격면역'] = '피격이상 면역' in sv
                        skill[f'skill{skill_count}_상태이상면역'] = '상태이상 면역' in sv
    except Exception as e:
        print(e)

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
list index out of range


# ArmoryGem

In [121]:
df = pd.read_csv('./data/ArmoryGem.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [30]:
gem_parts = [f'gem{i}' for i in range(1, 12)] 
dic_keys = ['type', 'level', 'grade','skillName', 'tier', 'availableTrade']
gems = {
    'characterCode': None,
}
for part in gem_parts:
    for key in dic_keys:
        gems[f'{part}_{key}'] = None

In [168]:
for i in range(100):
    gem = gems.copy()
    gem['characterCode'] = df['characterCode'][i]
    gem_count = 0
    try:
        cdf = eval(df['data'][i])['Gems']
        for j in range(len(cdf)):
            gem_count += 1
            fd = flatten_dict(cdf[j])
            gem[f'gem{gem_count}_grade'] = fd['Grade']
            gem[f'gem{gem_count}_level'] = fd['Level']
            for k, v in fd.items():
                sv = str(v)
                if '홍염' in sv:
                    gem[f'gem{gem_count}_type'] = '홍염'
                elif '멸화' in sv:
                    gem[f'gem{gem_count}_type'] = '멸화'
                elif '티어' in sv:
                    gem[f'gem{gem_count}_tier'] = int(int_pattern.findall(sv)[0])
                elif '재사용' in sv:
                    gem[f'gem{gem_count}_skillName'] =  sv.split('재사용')[0]
                elif '피해' in sv:
                    gem[f'gem{gem_count}_skillName'] =  sv.split('피해')[0]
                elif '거래' in sv:
                    gem[f'gem{gem_count}_availableTrade'] = '가능' in sv     
    except Exception as e:
        print(e)
        

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

# ArmoryEngraving

In [8]:
df = pd.read_csv('./data/ArmoryEngraving.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [43]:
engraving_parts = [f'engraving{i}' for i in range(1, 16)] 
dic_keys = ['name', 'level']
engravings = {
    'characterCode': None,
    'grantName1': None,'grantPoint1':None, 'grantName2':None, 'grantPoint2':None
}
for part in engraving_parts:
    for key in dic_keys:
        engravings[f'{part}_{key}'] = None

In [35]:
for i in range(100):
    engraving = engravings.copy()
    engraving['characterCode'] = df['characterCode'][i]
    engraving_count = 0
    grant_count = 0
    try:
        cdf = eval(df['data'][i])['Effects']
        for j in range(len(cdf)):
            engraving_count += 1
            fd = flatten_dict(cdf[j])
            engraving[f'engraving{engraving_count}_name'] = fd['Name'].split(' Lv. ')[0]
            engraving[f'engraving{engraving_count}_level'] = fd['Name'].split(' Lv. ')[-1]

        
        cdf = eval(df['data'][i])['Engravings']
        for j in range(len(cdf)):
            grant_count += 1
            fd = flatten_dict(cdf[j])
            engraving[f'grantName{grant_count}'] = fd['Name']
            for k, v in fd.items():
                sv = str(v)
                if '각인 활성' in sv:
                    engraving[f'grantPoint{grant_count}'] = int(int_pattern.findall(sv)[0])

    except Exception as e:
        print(e)

# ArmoryCard

In [36]:
df = pd.read_csv('./data/ArmoryCard.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [82]:
card_parts = [f'card{i}' for i in range(1, 7)]
dic_keys = ['name', 'level']
cards = {
    'characterCode': None,
}
for part in card_parts:
    for key in dic_keys:
        cards[f'{part}_{key}'] = None
for i in range(1, 6):
    cards[f'setName{i}'] = None
    cards[f'setPoint{i}'] = None

In [108]:
for i in range(10000):
    card = cards.copy()
    card['characterCode'] = df['characterCode'][i]
    card_count = 0
    set_count = 0
    try:
        cdf = eval(df['data'][i])['Cards']
        for j in range(len(cdf)):
            card_count += 1
            fd = flatten_dict(cdf[j])
            card[f'card{card_count}_name'] = fd['Name']
            card[f'card{card_count}_level'] = fd['AwakeCount']

        cdf = eval(df['data'][i])['Effects']
        for j in range(len(cdf)):
            set_count += 1
            try:
                set_name = cdf[j]['Items'][-1]['Name'].split(' (')
            except:
                continue
            card[f'setName{set_count}'] = set_name[0]
            if len(set_name) > 1:
                card[f'setPoint{set_count}'] = int(int_pattern.findall(set_name[1])[0])
            else:
                card[f'setPoint{set_count}'] = 0

    except Exception as e:
        print(e)

# Collectibles

In [109]:
df = pd.read_csv('./data/Collectibles.csv')
df = df.rename(columns={'Unnamed: 0':'characterCode', '0':'data'})

In [110]:
pprint(eval(df['data'][101]))

[{'CollectiblePoints': [{'MaxPoint': 51, 'Point': 10, 'PointName': '아르테미스'},
                        {'MaxPoint': 37, 'Point': 0, 'PointName': '유디아'},
                        {'MaxPoint': 71, 'Point': 9, 'PointName': '루테란 서부'},
                        {'MaxPoint': 127, 'Point': 25, 'PointName': '루테란 동부'},
                        {'MaxPoint': 72, 'Point': 17, 'PointName': '애니츠'},
                        {'MaxPoint': 87, 'Point': 10, 'PointName': '아르데타인'},
                        {'MaxPoint': 83, 'Point': 42, 'PointName': '베른 북부'},
                        {'MaxPoint': 57, 'Point': 9, 'PointName': '슈샤이어'},
                        {'MaxPoint': 70, 'Point': 13, 'PointName': '로헨델'},
                        {'MaxPoint': 70, 'Point': 1, 'PointName': '욘'},
                        {'MaxPoint': 70, 'Point': 3, 'PointName': '페이튼'},
                        {'MaxPoint': 70, 'Point': 8, 'PointName': '파푸니카'},
                        {'MaxPoint': 25, 'Point': 1, 'PointName': '베른 남부'},
                 

In [ ]:
collectible_parts = [f'collectible{i}' for i in range(1, 16)] 
dic_keys = ['name', 'level']
collectibles = {
    'characterCode': None
}
for part in collectible_parts:
    for key in dic_keys:
        collectibles[f'{part}_{key}'] = None

In [ ]:
for i in range(100):
    collectible = collectibles.copy()
    collectible['characterCode'] = df['characterCode'][i]
    collectible_count = 0
    try:
        cdf = eval(df['data'][i])['Effects']
        for j in range(len(cdf)):
            collectible_count += 1
            fd = flatten_dict(cdf[j])
            collectible[f'collectible{collectible_count}_name'] = fd['Name'].split(' Lv. ')[0]

    except Exception as e:
        print(e)